In [76]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,BatchNormalization,Activation,AveragePooling2D,GlobalAveragePooling2D,concatenate,Dense

In [77]:
def inception_module(x,f1,f2,f3):
    conv1=Conv2D(f1,(1,1),padding='same',activation='relu')(x)
    conv3=Conv2D(f2,(3,3),padding='same',activation='relu')(x)
    conv5=Conv2D(f3,(5,5),padding='same',activation='relu')(x)
    pool = MaxPooling2D((3,3),strides=(1,1),padding='same')(x)
    out=concatenate([conv1,conv3,conv5,pool])
    return out

In [78]:
img_input=Input(shape=(299,299,3))
classes=1000
Weights_path = 'inception_v3_weights_tf_dim_ordering_tf_kernels.h5'
channel_axis=3

In [79]:
def conv2d_bn(x,filters,num_row,num_col,padding='same',strides=(1,1)):
    x=Conv2D(filters,(num_row,num_col),padding=padding,strides=strides)(x)
    x=BatchNormalization(axis=3,scale=False)(x)
    x=Activation('relu')(x)
    return x

## Building the Model

In [80]:
def inc_blocka(x):
    branch1x1=conv2d_bn(x,64,1,1)
    
    branch5x5=conv2d_bn(x,48,1,1)
    branch5x5=conv2d_bn(branch5x5,64,3,3)
    
    branch3x3=conv2d_bn(x,64,1,1)
    branch3x3=conv2d_bn(branch3x3,96,3,3)
    branch3x3=conv2d_bn(branch3x3,96,3,3)

    maxpool3x3=AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool3x3=conv2d_bn(maxpool3x3,32,1,1)
    out=concatenate([branch1x1,branch5x5,branch3x3,maxpool3x3])
    return out

In [81]:
def red_blocka(x):
    maxpool3x3=MaxPooling2D((3,3),strides=(2,2))(x)

    branch3x3=conv2d_bn(x,384,3,3,strides=(2,2),padding='valid')

    branch1x1=conv2d_bn(x,64,1,1)
    branch1x1=conv2d_bn(branch1x1,96,3,3)
    branch1x1=conv2d_bn(branch1x1,96,3,3,strides=(2,2),padding='valid')
    out=concatenate([maxpool3x3,branch3x3,branch1x1],axis=channel_axis)
    return out

In [82]:
def inc_blockb(x):
    branch1x1=conv2d_bn(x,192,1,1)

    pool3x3=MaxPooling2D((3,3),strides=(1,1),padding='same')(x)
    pool3x3=conv2d_bn(pool3x3,192,1,1)

    branch1x1b=conv2d_bn(x,128,1,1)
    branch1x1b=conv2d_bn(branch1x1b,128,1,7)
    branch1x1b=conv2d_bn(branch1x1b,192,7,1)

    branch1x1c=conv2d_bn(x,128,1,1)
    branch1x1c=conv2d_bn(branch1x1c,128,7,1)
    branch1x1c=conv2d_bn(branch1x1c,128,1,7)
    branch1x1c=conv2d_bn(branch1x1c,128,7,1)
    branch1x1c=conv2d_bn(branch1x1c,192,1,7)

    out=concatenate([branch1x1,pool3x3,branch1x1b,branch1x1c])
    return out

In [83]:
def red_blockb(x):
    maxpool3x3=MaxPooling2D((3,3),strides=(2,2))(x)

    branch1x1=conv2d_bn(x,192,1,1)
    branch1x1=conv2d_bn(branch1x1,320,3,3,strides=(2,2),padding='valid')

    branch1x1b=conv2d_bn(x,192,1,1)
    branch1x1b=conv2d_bn(branch1x1b,192,1,7)
    branch1x1b=conv2d_bn(branch1x1b,192,7,1)
    branch1x1b=conv2d_bn(branch1x1b,192,3,3,strides=(2,2),padding='valid')

    out = concatenate([maxpool3x3,branch1x1,branch1x1b])
    return out

In [84]:
def inc_blockc(x):
    branch1x1=conv2d_bn(x,320,1,1)

    branch1x1b=conv2d_bn(x,384,1,1)
    branch1x1b_1=conv2d_bn(branch1x1b,384,1,3)
    branch1x1b_2=conv2d_bn(branch1x1b,384,3,1)
    branch1x1b=concatenate([branch1x1b_1,branch1x1b_2],axis=channel_axis)

    branch1x1c=conv2d_bn(x,448,1,1)
    branch1x1c=conv2d_bn(branch1x1c,384,3,3)
    branch1x1c_1=conv2d_bn(branch1x1c,384,3,1)
    branch1x1c_2=conv2d_bn(branch1x1c,384,1,3)
    branch1x1c=concatenate([branch1x1c_1,branch1x1c_2],axis=channel_axis)

    pool3x3=MaxPooling2D((3,3),strides=(1,1),padding='same')(x)
    pool3x3=conv2d_bn(pool3x3,192,1,1)

    out=concatenate([branch1x1,branch1x1b,branch1x1c,pool3x3])
    return out

In [85]:
x=conv2d_bn(img_input,32,3,3,padding='valid',strides=(2,2))
x=conv2d_bn(x,32,3,3,padding='valid')
x=conv2d_bn(x,64,3,3)
x=MaxPooling2D((3,3),strides=(2,2))(x)
x=conv2d_bn(x,80,1,1,padding='valid')
x=conv2d_bn(x,192,3,3,padding='valid')
x=MaxPooling2D((3,3),strides=(2,2))(x)
x=inc_blocka(x)
x=inc_blocka(x)
x=inc_blocka(x)
x=red_blocka(x)
x=inc_blockb(x)
x=inc_blockb(x)
x=inc_blockb(x)
x=inc_blockb(x)
x=red_blockb(x)
x=inc_blockc(x)
x=inc_blockc(x)
x=GlobalAveragePooling2D(name='avg_pool')(x)
x=Dense(classes,activation='softmax',name='predictions')(x)

inputs=img_input
model = keras.Model(inputs,x,name='inception_v3')
model.summary()

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)    │ (None, 299, 299, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_339 (Conv2D)           │ (None, 149, 149, 32)      │             896 │ input_layer_8[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_337       │ (None, 149, 149, 32)      │              96 │ conv2d_339[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_336 (Activation)   │ (None, 149, 149, 32)      │               0 │ batch_normalization_337[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_340 (Conv2D)           │ (None, 147, 147, 32)      │           9,248 │ activation_336[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_338       │ (None, 147, 147, 32)      │              96 │ conv2d_340[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_337 (Activation)   │ (None, 147, 147, 32)      │               0 │ batch_normalization_338[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_341 (Conv2D)           │ (None, 147, 147, 64)      │          18,496 │ activation_337[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_339       │ (None, 147, 147, 64)      │             192 │ conv2d_341[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_338 (Activation)   │ (None, 147, 147, 64)      │               0 │ batch_normalization_339[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_34              │ (None, 73, 73, 64)        │               0 │ activation_338[0][0]       │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_342 (Conv2D)           │ (None, 73, 73, 80)        │           5,200 │ max_pooling2d_34[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_340       │ (None, 73, 73, 80)        │             240 │ conv2d_342[0][0]           │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_339 (Activation)   │ (None, 73, 73, 80)        │               0 │ batch_normalization_340[0… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_343 (Conv2D)           │ (None, 71, 71, 192)       │         138,43

 Total params: 21,930,824 (83.66 MB)

 Trainable params: 21,898,056 (83.53 MB)

 Non-trainable params: 32,768 (128.00 KB)